In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import torch
from torch.optim import Adam
from torch.nn import functional as F
from torch.utils.data import DataLoader

import sys
sys.path.append("../")

from shared_utils.data import CSVPromptDataset
from early_exit.util import get_model
from shared_utils.load import get_tokenizer, configs_from_yaml
from shared_utils.generate import generate_text

from early_exit.patching import replace_attention_layers, set_transformer_early_exit_mode

# import wandb
import pandas as pd
import numpy as np

/usr/lib/python3/dist-packages/requests/__init__.py:87: RequestsDependencyWarning: urllib3 (2.5.0) or chardet (4.0.0) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "
/home/ubuntu/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2025-07-26 14:27:45.376495: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1753540065.395408    4560 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1753540065.401560    4560 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin 

In [3]:

# LOAD IN EXPERIMENT ARGS
# num_epoch = 1                     # args.num_epoch
num_exit_samples = 1                  # args.num_exit_samples
device = "cuda"                    # args.device
model_name = "deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B"                    # args.model_name
model_config_path = "./config_deepseek.yaml"                     # args.model_config_path
dataset_path = "./results_and_data/early_exit_sft_dataset/test/data.csv"                  # args.dataset_path
prompt_config_path = "./results_and_data/early_exit_sft_dataset/test/prompt_config.json"                    # args.prompt_config_path
batch_size = 1                    # args.batch_size -- might want to sort out batching, but increasi

In [4]:
# LOAD IN THE MODEL AND TOKENIZER
tokenizer = get_tokenizer(model_name)
config = configs_from_yaml(model_config_path, tokenizer.eos_token_id)
model = get_model(model_name, config['model'], device)


# LOAD IN DATASET
dataset = CSVPromptDataset(dataset_path, prompt_config_path)
dataloader = DataLoader(dataset, batch_size=batch_size, collate_fn=dataset.collate_fn, shuffle=True)


# ENABLE EARLY EXITING
model = replace_attention_layers(model, config['lora'], device)

replacing layer model.layers.0
replacing generate_layer_type_WITHOUT_early_exit_decision_head layer model.layers.1
replacing generate_layer_type_WITHOUT_early_exit_decision_head layer model.layers.2
replacing generate_layer_type_WITHOUT_early_exit_decision_head layer model.layers.3
replacing generate_layer_type_WITHOUT_early_exit_decision_head layer model.layers.4
replacing layer model.layers.5
replacing generate_layer_type_WITHOUT_early_exit_decision_head layer model.layers.6
replacing generate_layer_type_WITHOUT_early_exit_decision_head layer model.layers.7
replacing generate_layer_type_WITHOUT_early_exit_decision_head layer model.layers.8
replacing generate_layer_type_WITHOUT_early_exit_decision_head layer model.layers.9
replacing layer model.layers.10
replacing generate_layer_type_WITHOUT_early_exit_decision_head layer model.layers.11
replacing generate_layer_type_WITHOUT_early_exit_decision_head layer model.layers.12
replacing generate_layer_type_WITHOUT_early_exit_decision_head l

In [5]:
prompt = "Explain the concept of recursion in programming."
prompt = "Tell me a Zen joke about farmer"
system_prompt = "You are a helpful programming tutor."
prefiller = ""

set_transformer_early_exit_mode(model, 'sft_teacher')

with torch.no_grad():
    sft_teacher_response, (sft_teacher_generated_tokens, 
                          sft_teacher_final_layer_logprobs, 
                          gathered_early_exit_hidden_states) = generate_text(
        model=model,
        prompt=prompt,
        system_prompt=system_prompt,
        prefiller=prefiller,
        tokenizer=tokenizer,
        generation_config=config['generation'],
        device=device
    )
    
    early_output_log_probs = model.early_exit_hidden_state_readout(gathered_early_exit_hidden_states)
    
    early_exit_probs = model.early_exit_target_probs(
       early_output_log_probs=early_output_log_probs,
       teacher_final_layer_log_probs=sft_teacher_final_layer_logprobs
    )

transform_conversations currently only for Deepseek models!
full_tokenize currently only for Deepseek models!
prompt tokens shape: torch.Size([1, 20])


In [6]:
sft_teacher_response

'<｜begin▁of▁sentence｜><｜Assistant｜> You are a helpful programming tutor.\n<｜User｜> Tell me a Zen joke about farmer\n<｜Assistant｜> \nOkay, so I need to come up with a Zen joke about a farmer. Hmm, I\'m not super familiar with Zen philosophy, but I know it\'s about inner peace and mindfulness. So, the joke should probably have a twist or a twisty twist. Let me think about what a farmer does. They tend to work hard, maybe take risks, and deal with the world around them. \n\nMaybe I can play on the idea of the farmer being a hard worker but also a kind person. In Zen, there\'s the concept of the "soul in the earth," so maybe the farmer is the one who\'s in the earth, working hard, but also the one who\'s kind to the earth. \n\nLet me try to structure it. Start with something about the farmer, maybe a simple action like plowing the field. Then introduce the twist, perhaps something about the earth or the universe. Maybe the farmer is so focused on his work that he forgets the bigger picture